In [6]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Promijeni vidljivost <a href="javascript:code_toggle()">ovdje</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## Unutarnja stabilnost

Koncept stabilnosti bilježi ponašanje evolucije stanja sustava kada je isti "izbačen" iz ravnotežnog stanja: stabilnost opisuje divergira li evolucija stanja koja nastaje nakon poremećaja iz točke ravnoteže ili ne.

### Definicija
S obzirom na vremenski nepromjenjiv dinamički sustav opisan vektorom stanja $x(t)\in \mathbb{R}^n$, točkom ravnoteže $x_e$, početnim stanjem $x_0$ i početnim vremenom $t_0$, ako vrijedi

$$
\forall \, \epsilon \in \mathbb{R}, \, \epsilon > 0 \quad \exists \delta \in \mathbb{R}, \, \delta > 0 : \quad ||x_0-x_e|| < \delta \, \Rightarrow  \, ||x(t)-x_e|| < \epsilon \quad \forall t \ge t_0
$$
to bi se moglo protumačiti kao: ako postoji dovoljno mala početna perturbacija $\delta$ od točke ravnoteže takva da evolucija stanja $x(t)$ od točke poremećaja ne odlazi predaleko (više od $\epsilon$) od same ravnoteže, tada je točka ravnoteže stabilna.

Ako se također dogodi $\lim_{t\to\infty}||x(t)-x_e|| = 0$, što se može protumačiti kao: evolucija stanja se vraća natrag u točku ravnoteže, tada se kaže da je ravnoteža asimptotski stabilna.

U slučaju linearnih vremenski nepromjenjivih sustava:


\begin{cases}
\dot{x} = Ax +Bu \\
y = Cx + Du,
\end{cases}

moguće je dokazati da stabilnost jedne točke ravnoteže podrazumijeva stabilnost svih točaka ravnoteže, pa možemo govoriti o stabilnosti sustava čak i ako je, općenito, svojstvo stabilnosti povezano s točkom ravnoteže. Posebnost ovog linearnog sustava posljedica je činjenice da je evolucija ove vrste sustava strogo povezana sa svojstvenim vrijednostima matrice dinamike $A$, koje su invarijantne s obzirom na rotaciju, translaciju, početne uvjete i vrijeme.



Podsjetite se što je objašnjeno u primjeru o modalnoj analizi:

> Rješenje diferencijalne jednadžbe (u zatvorenoj formi), od početnog vremena $t_0$, s početnim uvjetima $x(t_0)$, je  
$$
x(t) = e^{A(t-t_0)}x(t_0).
$$ Matrica $e^{A(t-t_0)}x(t_0)$ se sastoji od linearnih kombinacija funkcije vremena $t$, svake tipa: $$e^{\lambda t},$$ gdje su $\lambda$-e svojstvene vrijednosti matrice $A$; ove su funckije modovi sustava.

stoga:
- linearni dinamički sustav stabilan je ako i samo ako svi njegovi modovi nisu divergentni,
- linearni dinamički sustav je asimptotski stabilan ako i samo ako su svi njegovi modovi konvergentni,
- linearni dinamički sustav je nestabilan ako ima barem jedan divergentni mod.

i, s obzirom na svojstvene vrijednosti matrice dinamike, to se događa ako:
- sve svojstvene vrijednosti matrice $A$ pripadaju <u>zatvorenoj</u> lijevoj polovici kompleksne ravnine (tj. realna vrijednost im je negativna ili nula), a, u slučaju da imaju realnu vrijednost nula, njihova je algebarska višestrukost ista kao geometrijska višestrukost (množnost), ili, ekvivalentno tome, imaju skalarne blokove u Jordanovom obliku;
- sve svojstvene vrijednosti pripadaju <u>otvorenoj</u> lijevoj polovici imaginarne ravnine, odnosno imaju strogo negativne realne dijelove;
- barem jedna svojstvena vrijednost ima pozitivan realni dio ili postoje svojstvene vrijednosti s realnom vrijednošću 0 i neskalarni Jordanovi blokovi.


Ovaj interaktivni primjer predstavlja matricu dinamike $A$ koju je moguće uređivati, a prikazuje slobodni odziv sustava uz odgovarajuće svojstvene vrijednosti.



### Kako koristiti ovaj interaktivni primjer?
- Pokušajte promijeniti svojstvene vrijednosti i početni uvjet $x_0$ i pogledajte kako se mijenja odziv.

In [7]:
%matplotlib inline
#%matplotlib notebook 
import control as control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [8]:
# Preparatory cell

A = numpy.matrix([[0,1],[-2/5,-1/5]])
X0 = numpy.matrix('5; 3')

Aw = matrixWidget(2,2)
Aw.setM(A)
X0w = matrixWidget(2,1)
X0w.setM(X0)

In [9]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

In [10]:
# Main cell

def main_callback(A, X0, DW):
    sols = numpy.linalg.eig(A)
    sys = sss(A,[[1],[0]],[0,1],0)
    pole = control.pole(sys)
    if numpy.real(pole[0]) != 0:
        p1r = abs(numpy.real(pole[0]))
    else:
        p1r = 1
    if numpy.real(pole[1]) != 0:
        p2r = abs(numpy.real(pole[1]))
    else:
        p2r = 1
    if numpy.imag(pole[0]) != 0:
        p1i = abs(numpy.imag(pole[0]))
    else:
        p1i = 1
    if numpy.imag(pole[1]) != 0:
        p2i = abs(numpy.imag(pole[1]))
    else:
        p2i = 1
    
    print('Svojstvene vrijednosti matrice A su:',round(sols[0][0],4),'i',round(sols[0][1],4))
    
    #T = numpy.linspace(0, 60, 1000)
    T, yout, xout = control.initial_response(sys,X0=X0,return_x=True)
    
    fig = plt.figure("Svojstvene vrijednosti od A", figsize=(16,16))
    ax = fig.add_subplot(311,title='Polovi (Re vs Img)')
    #plt.axis(True)
    # Move left y-axis and bottim x-axis to centre, passing through (0,0)
    # Eliminate upper and right axes
    ax.spines['left'].set_position(('data',0.0))
    ax.spines['bottom'].set_position(('data',0.0))
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.set_xlim(-max([p1r+p1r/3,p2r+p2r/3]),
                max([p1r+p1r/3,p2r+p2r/3]))
    ax.set_ylim(-max([p1i+p1i/3,p2i+p2i/3]),
                max([p1i+p1i/3,p2i+p2i/3]))
    
    plt.plot([numpy.real(pole[0]),numpy.real(pole[1])],[numpy.imag(pole[0]),numpy.imag(pole[1])],'o')
    plt.grid()

    ax1 = fig.add_subplot(312,title='Slobodni odziv')
    plt.plot(T,xout[0])
    plt.grid()
    ax1.set_xlabel('vrijeme [s]')
    ax1.set_ylabel('$x_1$')
    ax1.axvline(x=0,color='black',linewidth='0.8')
    ax1.axhline(y=0,color='black',linewidth='0.8')
    ax2 = fig.add_subplot(313)
    plt.plot(T,xout[1])
    plt.grid()
    ax2.set_xlabel('vrijeme [s]')
    ax2.set_ylabel('$x_2$')
    ax2.axvline(x=0,color='black',linewidth='0.8')
    ax2.axhline(y=0,color='black',linewidth='0.8')
    
    #plt.show()
    

    
alltogether = widgets.HBox([widgets.VBox([widgets.Label('$A$:',border=3),
                                          Aw]),
                                          widgets.Label('   ',border=3),
                            widgets.VBox([widgets.Label('$X_0$:',border=3),
                                          X0w]),
                                          START])
out = widgets.interactive_output(main_callback, {'A':Aw, 'X0':X0w, 'DW':DW})
out.layout.height = '1000px'
display(out, alltogether)

Output(layout=Layout(height='1000px'))